<font color='black'>
    <h1>PySpark DataFrame Processing Examples</h1>
</font>

<font color='steelblue'>

<span style="font-family:Arial; font-size:1.6em;">
<strong>PySpark Examples that shows:</strong><br>
<ol>
        <li>sort and orderBy</li>
        <li>withColumn</li>
        <li>Aggregate and groupBy</li>
        <li>filter</li>
        <li>nested structures in dataframe</li>
        <li>write and read parquet files</li>
        <li>read machine learning model that was saved to parquet file</li>
        <li>example of reading a JSON file</li>
    </ol>    
</span>

</font>

In [ ]:
# Set up the environment for using pyspark
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext

In [ ]:
# Create Application Context
spark = SparkSession.builder.appName("PySpark DataFrame Examples").getOrCreate()
sc = spark.sparkContext

In [ ]:
sc.setLogLevel("Error")

In [ ]:
# Data type definitions
from pyspark.sql.types import *

# Lot of functions used in this notebook
from pyspark.sql.functions import *

# sort and orderBy examples

In [ ]:
# List of python tuples
df = spark.createDataFrame([("James","M", "Sales","TX",90000,34,10000), \
    ("Michael","M", "Sales","TX",86000,56,20000), \
    ("Robert","M","Sales","CA",81000,30,23000), \
    ("Maria","F","Finance","CA",90000,24,23000), \
    ("John","M","Finance","CA",99000,40,24000), \
    ("Anna","F","Finance","TX",83000,36,19000), \
    ("Jen","F","Finance","TX",79000,53,15000), \
    ("Jeff","M","Marketing","CA",80000,25,18000), \
    ("Anthony","M","Marketing","TX",91000,50,21000) \
  ], ["employee_name","gender","department","state","salary","age","bonus"])

In [ ]:
# Show the schema for the dataframe
df.printSchema()

In [ ]:
df.show(truncate = False)

In [ ]:
# sort by department and then state
df.sort("department","state").show(truncate=False)

# can also do it using col - same results
# df.sort(col("department"), col("state")).show(truncate = False)

In [ ]:
# Using orderBy on department and then state (descending order) [Use asc for ascending order]
df.orderBy(col("department").desc(),col("state").desc()).show(truncate=False)

# withColumn examples

In [ ]:
data = [(("James","E","Jones"),"44221","M","3500"), \
      (("Michael","Rose","Smith"),"39456","M","3000"), \
      (("Robin","","Williams"),"45912","M","4000"), \
      (("Maria","A","Edwards"),"39192","F","4000"), \
      (("Jen","C","Brown"),"","F","-1") \
]

In [ ]:
mySchema = StructType([
        StructField('name', StructType([
            StructField('firstname', StringType(), True),
            StructField('middlename', StringType(), True),
            StructField('lastname', StringType(), True)
        ])),
        StructField('dob', StringType(), True),
        StructField('gender', StringType(), True),
        StructField('salary', StringType(), True)
        ])

In [ ]:
wdf = spark.createDataFrame(data = data, schema = mySchema)

In [ ]:
wdf.show(truncate = False)

In [ ]:
wdf.printSchema()

In [ ]:
# Create new dataframe and change the datatype of salary to integer
df1 = wdf.withColumn("salary",col("salary").cast("Integer"))
df1.show(truncate = False)

In [ ]:
df1.dtypes

In [ ]:
# Change an existing column
df2 = df1.withColumn("salary",col("salary") * 100)
df2.show(truncate = False)

In [ ]:
# Create a new column (newSalary) from an existing column
df3 = df2.withColumn("newSalary",col("salary")* -1)
df3.show(truncate = False)

In [ ]:
# Add a new column to an dataframe and set a value for the whole column
df4 = df3.withColumn("country", lit("USA"))
df4.show(truncate = False)

In [ ]:
# Rename a column
df5 = df4.withColumnRenamed("gender","sex")
df5.show(truncate = False)

In [ ]:
# Drop a column
df5 = df5.drop('newSalary')
df5.show(truncate = False)

# Aggregates Examples

In [ ]:
df.show()

In [ ]:
# approx_count_distinct() function returns the count of distinct items

# collect gives a Row object which is embedded in a python list (first index gives the row object 
# and then the second gives the data )

print("salary: approx_count_distinct: {}".format(df.select(approx_count_distinct("salary")).collect()[0][0]))

# average salary
print("Average salary: {}".format(df.select(avg("salary")).collect()[0][0]))

In [ ]:
# countDistinct() returns the number of distinct elements in a columns
df1 = df.select(countDistinct("department", "salary"))
print("Distinct in department and salary: {}".format(df1.collect()[0][0]))

In [ ]:
# count
df.select(count('age')).collect()[0][0]

In [ ]:
# get the mean of age - also supported are max(), min(), stddev(), variance()
df.select(mean("age")).show(truncate=False)

# groupBy examples

In [ ]:
df.show()

In [ ]:
# Take the sum of sales by department
df.groupBy("department").sum("salary").show(truncate=False)

In [ ]:
# Count distinct departments using count(), can also do min(), max(), avg()
df.groupBy("department").count().collect()

In [ ]:
# get mean per department of age, salary and bonus (avg() will yield same results)
df.groupBy("department").mean().collect()

In [ ]:
# Group by department and state and then take sum for each take the sum for salary and bonus
df.groupBy("department","state").sum("salary","bonus").show(truncate = False)

In [ ]:
# Get aggregates of multiple columns grouped by department
df.groupBy("department") \
    .agg(sum("salary").alias("sum_salary"), \
         avg("salary").alias("avg_salary"), \
         sum("bonus").alias("sum_bonus"), \
         max("bonus").alias("max_bonus") \
     ) \
    .show(truncate=False)

In [ ]:
# Similar to SQL “HAVING” clause, On PySpark DataFrame we can use either where() or 
# filter() function to filter the rows of aggregated data.
# Remove any bonus sum less than 50000

df.groupBy("department") \
    .agg(sum("salary").alias("sum_salary"), \
      avg("salary").alias("avg_salary"), \
      sum("bonus").alias("sum_bonus"), \
      max("bonus").alias("max_bonus")) \
    .where(col("sum_bonus") >= 50000) \
    .show(truncate=False)

# filter examples

In [ ]:
# get all females in dataframe
df.filter(df.gender == 'F').show(truncate = False)

In [ ]:
# filter with multiple conditions
df.filter((df.gender == 'F') & (df.state == 'TX')).show(truncate = False)

# Nested structures examples

In [ ]:
df1 = spark.createDataFrame([
  ("James",[["Bread","Butter","Meat"],["Diet Coke","Chips"]]),
  ("Michael",[["Butter","Bread","Fruits"],["Mineral Water","Nuts"]]),
  ("Robert",[["Bread","Butter"],["Pepsi","Cookies"]])
], ['name', 'groceries'])

In [ ]:
df1.printSchema()

In [ ]:
df1.show(truncate=False)

In [ ]:
# explode “groceries” array column to array rows. after exploding, 
# it creates a new column ‘col’ with rows represents an array.

df1.select(df1.name,explode(df1.groceries)).show(truncate = False)

In [ ]:
# use flatten function which converts array of array columns to a single array on DataFrame
df1.select(df1.name,flatten(df1.groceries)).show(truncate = False)

# Parquet format

In [ ]:
# Following code is to make sure that the directory is deleted
# before writing the parquet file

import os
import sys
import shutil

# Get directory name
mydir = 'myDataFrame'

## Try to remove tree; if failed show an error using try...except on screen
try:
    shutil.rmtree(mydir)
except OSError as e:
    print ("Error: %s - %s." % (e.filename, e.strerror))

In [ ]:
# write the dataframe to a parquet file
df.write.parquet('myDataFrame')

In [ ]:
ndf = spark.read.parquet('myDataFrame')

In [ ]:
ndf.show(truncate = False)

<font color='steelblue'>
    <h1>Load parquet file</h1>
        <h2>File contains a linear regression model information</h2>
</font>

In [ ]:
model = spark.read.parquet('../datasets/linearRegModel.parquet')

In [ ]:
print(type(model))

In [ ]:
model.show(truncate = False)

## Pandas dataframe

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
pdf = model.toPandas()
pdf.head()

<font color='steelblue'>
    <h1>Load JSON file</h1>
        <h2>File contains information about IRIS flowers</h2>
</font>

In [ ]:
irisdf = spark.read.json('../datasets/iris.json')

In [ ]:
irisdf.show(truncate = False)

In [ ]:
print("rows: {} columns: {}".format(irisdf.count(), len(irisdf.columns)))

In [ ]:
irisdf.printSchema()

In [ ]:
print(type(irisdf))

In [ ]:
irisdf.groupBy('species').count().collect()

In [ ]:
irisdf.describe().show()

In [ ]:
irisdf.describe()